In [19]:
from datasets import Dataset
import pandas as pd
import torch, csv, json
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import bitsandbytes as bnb
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig
from datasets import load_dataset
from dotenv import load_dotenv

In [20]:
load_dotenv()

True

In [21]:
def csv_to_jsonl(csv_path, jsonl_path):
    with open('ttt_prompt.txt', 'r') as file:
        # Read the entire file into a string
        prompt = file.read()
    with open(csv_path, 'r') as csv_file, open(jsonl_path, 'w') as jsonl_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            jsonl_file.write(
                json.dumps({"prompt": prompt.format(state = row["Game States"]), "completion": f'{{\n\tnext_best_move: {row["Optimal Moves"]}\n}}'}) + '\n')

In [22]:
# Load the csv file into a pandas DataFrame
df = pd.read_csv('../examples/ttt_data.csv')
csv_to_jsonl('../examples/ttt_data.csv', "data.jsonl")

In [23]:
dataset = load_dataset("json", data_files="data.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [24]:
print(dataset[0]['prompt'])

You are a tic-tac-toe solver. A tic-tac-toe board is a 3x3 grid. For example

b,o,b
x,b,b
b,b,o

b represents an empty position
o represents a mark by player 1
x represents a mark by player 2

This state can also be represented in one line eg.
bobxbbbbo

The grid is also numbered where each number represents a position on the grid. eg.
1,2,3
4,5,6
7,8,9

a move can thus be represented by mark+number. Here are some examples:
o5 means player 1 marks position 5 on the grid
x1 means player 2 marks positoin 4 on the grid

Your job is to generate the next best move given a tic-tac-toe board state.

Your output should be in the following json format:
{
    next_best_move: <move>
}

Given the following state, what is the next best move?
bbbxoobox


In [25]:
print(dataset[0]['completion'])

{
	next_best_move: x2
}


In [26]:
device = "cuda:0"

LR = 1.41e-5
BATCH_SIZE = 1
MINI_BATCH_SIZE = 1

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model_id = "google/gemma-2b-it"

lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=device)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    packing=True,
    peft_config=lora_config
)
trainer.train()
trainer.save_pretrained("gemma-2b-it-sft")

/home/heyandy/venv/rlproject/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

/home/heyandy/venv/rlproject/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/heyandy/venv/rlproject/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/home/heyandy/venv/rlproject/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/heyandy/venv/rlproject/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/heyandy/venv/rlproject/lib/python3.8/site-packages/peft/peft_model.py", line 1129, in forward
    return self.base_model(
  File "/home/heyandy/venv/rlproject/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/heyandy/venv/rlproject/lib/python3.8/site-packages/torch/nn/modules/module.